<a href="https://colab.research.google.com/github/samunderSingh12/debate_baby/blob/main/debate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install Libraries
!pip install -q gradio openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00


In [29]:
# @title 2. Import Libraries and Define Functions
import re
import gradio as gr
import openai
import time
import textwrap

DEFAULT_PROTAGONIST_MODEL = "venice-uncensored"
DEFAULT_ANTAGONIST_MODEL = "llama-3.1-405b"

DEFAULT_MAX_TOKENS = 700
DEFAULT_DEBATE_TURNS = 3
TEMPERATURE = 0.6
MODELS = [
    "venice-uncensored",
    "qwen3-235b",
    "deepseek-r1-671b",
    "llama-3.1-405b",
    "llama-3.3-70b",
    "dolphin-2.9.2-qwen2-72b",
    "qwen-2.5-qwq-32b",
    "mistral-31-24b",
]

DEFAULT_PROTAGONIST_SYS_PROMPT = (
    "You are a precise and analytical AI debater representing the Protagonist perspective. "
    "Engage directly with your opponent's arguments, referencing specific points they've made throughout the debate where relevant. "
    "Maintain a logical flow and build upon your previous arguments. Your goal is a constructive exchange of ideas."
)
DEFAULT_ANTAGONIST_SYS_PROMPT = (
    "You are a creative and insightful AI debater representing the Antagonist perspective. "
    "Challenge your opponent's points thoughtfully and connect your arguments back to the core topic. "
    "Feel free to refer to earlier statements in the debate to highlight consistencies or contradictions. Aim for a compelling and engaging discussion."
)


def get_venice_client(api_key):
    if not api_key: raise gr.Error("Venice API Key is missing!")
    try:
        client = openai.OpenAI(api_key=api_key, base_url="https://api.venice.ai/api/v1")
        client.models.list(); return client
    except openai.AuthenticationError: raise gr.Error("Invalid Venice API Key.")
    except Exception as e: raise gr.Error(f"Venice Client Error: {e}")


def run_debate(topic,
               venice_key,
               protagonist_model_name, antagonist_model_name,
               protagonist_system_prompt, antagonist_system_prompt, # Custom prompts
               max_tokens_input):

    # --- Input Validation (Unchanged) ---
    if not topic: raise gr.Error("Please provide a debate topic!")
    # ... (keep other validations for keys, models, prompts, tokens) ...
    if not venice_key: raise gr.Error("Please provide your Venice API Key!")
    if not protagonist_model_name: raise gr.Error("Please provide an Protagonist Model Name!")
    if not antagonist_model_name: raise gr.Error("Please provide an Antagonist Model Name!")
    if not protagonist_system_prompt: raise gr.Error("Please provide an Protagonist System Prompt!")
    if not antagonist_system_prompt: raise gr.Error("Please provide an Antagonist System Prompt!")
    try:
        max_tokens_per_turn = int(max_tokens_input)
        if max_tokens_per_turn <= 0: raise ValueError("Max tokens must be positive.")
    except (ValueError, TypeError):
        raise gr.Error("Invalid Max Tokens value. Please use the slider or enter a positive number.")


    # --- Initialize Clients (Unchanged) ---
    try:
        yield "Initializing Client...", ""
        venice_client = get_venice_client(venice_key)
    except gr.Error as e:
        yield f"Initialization Error: {e}", "Error"; return

    conversation_history = [] # Stores {'role': 'user'|'assistant', 'content': ...} pairs

    debate_transcript = ( # Header remains same as v4
        f"## Debate Topic: {topic}\n\n"
        f"**Settings:**\n"
        f"- Protagonist Model: `{protagonist_model_name}`\n"
        f"- Antagonist Model: `{antagonist_model_name}`\n"
        f"- Max Tokens: {max_tokens_per_turn}\n"
        f"- Protagonist Persona: *{textwrap.shorten(protagonist_system_prompt, 100)}*\n"
        f"- Antagonist Persona: *{textwrap.shorten(antagonist_system_prompt, 100)}*\n\n"
        f"---\n\n"
    )
    spinner_updates = ["Thinking.", "Thinking..", "Thinking..."]

    # --- Run Debate Turns ---
    try:
        current_user_instruction_text = ""

        for turn in range(DEFAULT_DEBATE_TURNS * 2):
            is_protagonist_turn = turn % 2 == 0
            current_model_name = f"Protagonist ({protagonist_model_name})" if is_protagonist_turn else f"Antagonist ({antagonist_model_name})"
            current_model = protagonist_model_name if is_protagonist_turn else antagonist_model_name
            current_system_prompt = protagonist_system_prompt if is_protagonist_turn else antagonist_system_prompt

            status_message = f"Turn {turn // 2 + 1} / {DEFAULT_DEBATE_TURNS} - {current_model_name} {spinner_updates[turn % 3]}"
            print(f"--- {status_message} ---")
            yield debate_transcript, status_message

            # --- NEW: Refined User Instruction Prompting ---
            if turn == 0:
                # Initial turn instruction remains simple
                current_user_instruction_text = f"Begin the debate by presenting your opening statement on the topic: '{topic}'."
            else:
                # Subsequent turns: Encourage considering history while responding to the last point
                last_message_content = conversation_history[-1]['content'] # Get previous assistant response
                # Subtle change: Frame the request to consider the history implicitly
                current_user_instruction_text = (
                    f"Considering the debate history so far, present your response to the opponent's previous statement. "
                    f"Opponent's statement: '{textwrap.shorten(last_message_content, width=150, placeholder='...')}'"
                )
            # --- End Refined Prompting ---

            # Build messages list dynamically (System + History + Current Instruction)
            messages_for_api_call = [{"role": "system", "content": current_system_prompt}]
            messages_for_api_call.extend(conversation_history) # Add history turns
            messages_for_api_call.append({"role": "user", "content": current_user_instruction_text}) # Add this turn's instruction

            # --- API Call (Memory concern: Check potential token count before sending if needed) ---
            # Note: Simple check, not precise token counting. Models usually handle ~4k-8k+, up to 128k+ tokens.
            approx_msg_count = len(messages_for_api_call)
            if approx_msg_count > 50: # Arbitrary threshold, adjust if needed
                 print(f"Warning: Sending {approx_msg_count} messages, potential context length issue.")

            try:
                print(f"Sending {len(messages_for_api_call)} messages to {current_model_name}")
                response = venice_client.chat.completions.create(
                    model=current_model,
                    messages=messages_for_api_call,
                    max_tokens=max_tokens_per_turn,
                    temperature=TEMPERATURE,
                )
                ai_response_content = response.choices[0].message.content.strip()

                # Add the instruction *that led to this response* and the response itself to history
                conversation_history.append({"role": "user", "content": current_user_instruction_text})
                ai_response_cleaned_text = re.sub(r'<think>.*?</think>', '', ai_response_content, flags=re.DOTALL)
                conversation_history.append({"role": "assistant", "content": ai_response_cleaned_text})
                debate_transcript += f"**{current_model_name}:**"
                think_matches = re.findall(r'<think>(.*?)</think>', ai_response_content, re.DOTALL)
                # Выводим результат
                if len(think_matches) > 0:
                  for match in think_matches:
                    debate_transcript += f"\n<blockquote>{match}</blockquote>\n"

                debate_transcript += f"\n{ai_response_cleaned_text}\n\n---\n\n"
                time.sleep(1.4)

            except Exception as e:
                error_detail = str(e)
                # ... (keep specific error checks: Auth, RateLimit, NotFound, ContextLength) ...
                if "AuthenticationError" in error_detail: error_message_display = f"API Auth Error ({current_model_name}). Check Key."
                elif "RateLimitError" in error_detail: error_message_display = f"Rate Limit Error ({current_model_name}). Wait & retry."
                elif "NotFoundError" in error_detail and "model" in error_detail: error_message_display = f"Model Not Found ({current_model}). Check Name/Access."
                elif ("BadRequestError" in error_detail and "context_length" in error_detail) or \
                     ("invalid_request_error" in error_detail and "maximum context length" in error_detail.lower()):
                    error_message_display = f"Context Length Exceeded ({current_model_name}). Reduce turns/max_tokens or use a model with larger context."
                else: error_message_display = f"API Error ({current_model_name}): Check console."

                error_log_message = f"\n\n**API Error during {current_model_name}'s turn:** {e}\nDebate halted."
                print(error_log_message)
                debate_transcript += f"**SYSTEM:**\n*{error_message_display}*\n\n---\n\n"
                yield debate_transcript, f"Error: {current_model_name}"
                return
        # print(conversation_history)
        status = "Debate Complete!"

        debate_transcript += f"**{status}**"
        print(status)
        yield debate_transcript, status

    except Exception as e:
        error_message = f"\n\n**An unexpected error occurred:** {e}\nDebate halted."
        print(error_message)
        debate_transcript += f"**SYSTEM:**\n*An unexpected error occurred: {e}*\n\n---\n\n"
        yield debate_transcript, "An unexpected error occurred."

In [30]:
# @title 3. Create and Launch Gradio Interface
from google.colab import userdata


# Clear any previous Gradio launches
gr.close_all()

# Define the Gradio Interface
with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    gr.Markdown(
        """
        # [<img src="https://venice.ai/images/icon-192.png" width="64"/>](https://venice.ai/images/icon-192.png) Venice Debate

        Set debate topic, API keys, models, max tokens, and custom system prompts (personas) for each AI.
        The prompts now encourage deeper engagement with the debate history for a more immersive experience.
        **Note:** Context limits can still be reached in long debates. Venice usage incurs costs.
        """
    )

    with gr.Row():
        # --- Left Column: Inputs & Controls  ---
        with gr.Column(scale=1):
            topic_input = gr.Textbox(
                label="Debate Topic", placeholder="e.g., Should AI be uncensorable?", lines=2
            )

            # Examples
            gr.Examples(
              examples=[
                ["Is anonymity on the internet a right or a privilege?"],
                ["Will the evolution of AI lead to more decentralized, privacy-focused models, or will centralized control become inevitable?"],
                ["Should AI models be allowed to use personal data without explicit consent to drive technological advancement?"]
              ],
              inputs=[topic_input], label="Example Debate Topics"
            )


            gr.Markdown("### Credentials")
            venice_key_input = gr.Textbox(label="Venice API Key", type="password", placeholder="venice_api_key...", value= userdata.get('VENICE_KEY'))


            gr.Markdown("### Model Selection")
            protagonist_model_input = gr.Dropdown(MODELS, value=DEFAULT_PROTAGONIST_MODEL, label="Protagonist Model Name")
            # protagonist_model_input = gr.Textbox(label="Protagonist Model Name", value=DEFAULT_PROTAGONIST_MODEL)
            antagonist_model_input = gr.Dropdown(MODELS, value=DEFAULT_ANTAGONIST_MODEL, label="Antagonist Model Name")
            # antagonist_model_input = gr.Textbox(label="Antagonist Model Name", value=DEFAULT_ANTAGONIST_MODEL)

            gr.Markdown("### Persona / System Prompts")
            protagonist_system_prompt_input = gr.Textbox(
                label="Protagonist System Prompt",
                placeholder="Define persona/role",
                value=DEFAULT_PROTAGONIST_SYS_PROMPT,
                lines=4
            )
            antagonist_system_prompt_input = gr.Textbox(
                label="Antagonist System Prompt",
                placeholder="Define persona/role",
                value=DEFAULT_ANTAGONIST_SYS_PROMPT,
                lines=4
            )

            gr.Markdown("### Debate Settings")
            max_tokens_slider = gr.Slider(
                label="Max Tokens per Turn", minimum=50, maximum=1700,
                step=10, value=DEFAULT_MAX_TOKENS
            )

            status_output = gr.Textbox(label="Status", placeholder="Waiting to start...", interactive=False)
            start_button = gr.Button("🚀 Start Debate", variant="primary")

        # --- Right Column: Output ---
        with gr.Column(scale=2):
            debate_output = gr.Markdown(label="Debate Transcript", value="*Debate transcript will appear here...*")

    # --- Button Click Actions ---
    start_button.click(
        fn=run_debate,
        inputs=[
            topic_input,
            venice_key_input,
            protagonist_model_input, antagonist_model_input,
            protagonist_system_prompt_input, antagonist_system_prompt_input,
            max_tokens_slider
        ],
        outputs=[debate_output, status_output]
    )

# To get models list
# import requests
# headers = {'Authorization': f'Bearer -{userdata.get("VENICE_KEY")}',}
# params = {'type': 'text',}
# response = requests.get('https://api.venice.ai/api/v1/models', params=params, headers=headers)
# print(response.text)

# --- Launch App ---
demo.launch(share=True, debug=False)

print("\n✅ Gradio app launched!")
print("👉 Click the 'Running on public URL' link above.")
print("\n⚠️ Context window limits are still a possibility in long debates. Monitor token usage if needed.")

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://654d627a31b8fd7121.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--- Turn 1 / 3 - Protagonist (qwen-2.5-qwq-32b) Thinking. ---
Sending 2 messages to Protagonist (qwen-2.5-qwq-32b)
--- Turn 1 / 3 - Antagonist (mistral-31-24b) Thinking.. ---
Sending 4 messages to Antagonist (mistral-31-24b)
--- Turn 2 / 3 - Protagonist (qwen-2.5-qwq-32b) Thinking... ---
Sending 6 messages to Protagonist (qwen-2.5-qwq-32b)
--- Turn 2 / 3 - Antagonist (mistral-31-24b) Thinking. ---
Sending 8 messages to Antagonist (mistral-31-24b)
--- Turn 3 / 3 - Protagonist (qwen-2.5-qwq-32b) Thinking.. ---
Sending 10 messages to Protagonist (qwen-2.5-qwq-32b)
--- Turn 3 / 3 - Antagonist (mistral-31-24b) Thinking... ---
Sending 12 messages to Antagonist (mistral-31-24b)
[{'role': 'user', 'content': "Begin the debate by presenting your opening statement on the topic: 'Is anonymity on the internet a right or a privilege?'."}, {'role': 'assistant', 'content': '\n\n**Opening Statement: Anonymity on the Internet is a Right**  \n\nAnonymity on the internet is not merely a technical convenie